In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import os, sys

def get_dir_n_levels_up(path, n):
    # Go up n levels from the given path
    for _ in range(n):
        path = os.path.dirname(path)
    return path


proj_root = get_dir_n_levels_up(os.path.abspath("__file__"), 2)
sys.path.append(proj_root)

import traceback
from typing import Dict

from liftoff import parse_opts


from opinion_dqn import AgentDQN
from utils import my_logging
from utils.experiment import seed_everything, create_path_to_experiment_folder, build_environment
from utils.generic import convert_namespace_to_dict
import yaml

In [3]:
# softmax in last layer because it scales with N instead of 2^N where N is the nr of agents

# might need to do policy gradient

# next step: Q iteration with action representation

In [6]:
yaml_path = r"D:\Work\repos\RL\phd-rl-algos\dynamic_programming\opinion_dynamics\dqn\experiments\results\2025Jan14-225523_configs\0000_estimator.args_.lin_hidden_out_size_128__estimator.args_.conv_hidden_out_size_32\0\cfg.yaml"

with open(yaml_path, 'r') as file:
    config = yaml.safe_load(file)
seed = int(os.path.basename(config["out_dir"]))

seed_everything(seed)

logs_file = os.path.join(config["out_dir"], "experiment_log.log")

logger = my_logging.setup_logger(
    name=config["experiment"],
    # log_file=logs_file,
)

logger.info(f"Starting experiment: {config['full_title']}")

### Setup environments ###
train_env = build_environment()
validation_env = build_environment()

### Setup output and loading paths ###

path_previous_experiments_outputs = None
if "restart_training_timestamp" in config:
    path_previous_experiments_outputs = create_path_to_experiment_folder(
        config,
        config["out_dir"],
        config["restart_training_timestamp"],
    )

experiment_agent = AgentDQN(
    train_env=train_env,
    validation_env=validation_env,
    experiment_output_folder=config["out_dir"],
    experiment_name=config["experiment"],
    resume_training_path=path_previous_experiments_outputs,
    save_checkpoints=True,
    logger=logger,
    config=config
)

logger.info(
    f'Initialized agent with models: {experiment_agent.policy_model}'
)

experiment_agent.train(train_epochs=config["epochs_to_train"])

logger.info(
    f'Finished training experiment: {config["full_title"]}, seed: {config["seed"]}'
)

my_logging.cleanup_file_handlers(experiment_logger=logger)




2025-01-14 23:17:55,868 - opinion_agent_dqn - INFO - Starting experiment: 2025Jan14-225523_configs_estimator.args_.lin_hidden_out_size=128; estimator.args_.conv_hidden_out_size=32
2025-01-14 23:17:55,868 - opinion_agent_dqn - INFO - Starting experiment: 2025Jan14-225523_configs_estimator.args_.lin_hidden_out_size=128; estimator.args_.conv_hidden_out_size=32
2025-01-14 23:17:55,868 - opinion_agent_dqn - INFO - Starting experiment: 2025Jan14-225523_configs_estimator.args_.lin_hidden_out_size=128; estimator.args_.conv_hidden_out_size=32


IndexError: tuple index out of range